# Ejemplo Web scrapping: Mobiliaria Casa Sapo


En este notebook vamos a ver algunos ejemplos para hacer web scrapping, que consiste en obtener cierta información publicada en alguna web. Para ello, nos basaremos en cómo está compuesta la web de la que queremos extraer información.

Todas las webs están diseñadas en HTML*, un lenguaje basado en etiquetas. No nos vamos a meter a conocer el lenguaje HTML aquí, lo único que nos importa de momento es saber que estos códigos son unas cuántas líneas de texto plano con diferentes elementos organizados con etiquetas. Con ellas se define si en una posición va un texto (y su formato), una imagen (que será un enlace a su dirección), un link que te lleve a otra dirección...

Con el *web scrapping* podremos atacar la parte diseñada en HTML. Básicamente, nuestra función será ir mirando las etquetas y quedándonos con aquellas que queramos. Para ello, tendremos que investigar cómo está hecha la web, definir qué queremos "scrapear" e identificar el patrón de etiquetas bajo el que se encuentra. Cabe destacar, que el sistema de etiquetas es un sistema de árbol, donde un elemento identificado por una etiqueta podría contener otros elementos que tuvieran a su vez otras etiquetas.


\**Pese a que todos los sitios web estén basadas en HTML, casi siempre convive junto a otros scripts en otros lenguajes para darle la inteligencia a su web, como podrían ser scripts de Javascript o php. Básicamente, podemos interpretar que el HTML lleva la información y la estructura de la página, mientras que el resto de elementos que aportan funcionalidad a la web serán scripts complementarios en otros lenguajes, como un reproductor de vídeo (con HTML podré ubicarlo en la web pero la funcionalidad quedará de parte de otro lenguaje).*

## Importando librerías

Para obtener el HTML de una web, hay que saber gestionar lo protocolos de transferencia de datos, pero nosotros, al igual que el programador medio, no sabemos hacaerlo. ¿Cómo vamos a hacerlo entonces? Simple, utiliaremos librerías que se encarguen de ello por nosostros. En este caso, nos basaremos en ``requests`` para las conexiones con las webs, y en ``BeautifulSoup`` para analizar los HTML que extraigamos, de modo que nos sea mucho más sencillo detectar las etiquetas y navegar por ellas.

In [250]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()


from urllib.request import urlopen

En primer lugar, comenzamos definiendo una cabecera para el intercambio de información, es decir, para simular que somos un navegador y que podamos obtener el texto plano que define el HTML. Lo que añadamos en la cabecera lo vamos a mantener así y no vamos a meternos a ello, pero básicamente es para lo que acabamos de coemntar.

La solicitud de cabecera del Agente de Usuario contiene una cadena característica que permite identificar el protocolo de red que ayuda a descubrir  el tipo de aplicación, sistema operativo, provedor del software o la versión del software de la petición del agente de usuario.

In [251]:
headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})

Tras ello, definimos qué web queremos "scrapear" y pedimos su HTML (función ``get``) como si fuéramos un navegador web.

In [252]:
sapo = "https://casa.sapo.pt/comprar-apartamentos/cascais/"

response = get(sapo, headers = headers)

Con esta petición, obtendremos un objeto que hemos guardado en la variable response. Este objeto tiene una estructura diseñada específicamente para la conexión web, y si hacemos un print a lo bestia de él, nos devolverá un código.

Si obtenemos un ``200`` significa que todo ha ido bien, y que el objeto tendrña la información que he solicitado. Sin embargo, puede que devuelva otro código, en cuyo caso hay algo que debemos corregir, no tenemos acceso, hemos introducido mlas la dirección...

In [253]:
print(response) # un 200 es una buena señal :D

<Response [200]>


Como hemos obtenido un 200, significa que todo ha ido bien y que podremos accedera a las diferentes características del mismo, como el contenido de la web, las cabeceras...

A continuación, se muestra lo que se obtiene si hacemos un print de los primeros 1000 caracteres de texto (es el código HTML de la web que estamos atacando):

In [254]:
print(response.text[:1000])



<!DOCTYPE html>

<html lang="pt">
<head><title>
	Casas para Venda, Apartamentos em Cascais, CASA SAPO - Portal Nacional de Imobiliário
</title><meta name="author" content="CASA SAPO - Portal Nacional de Imobiliário - Janela Digital SA" />
    <meta name="application-name" content="CASA SAPO - Portal Nacional de Imobiliário" data-copyright="Janela Digital SA" data-generated-time="21/01/2021 17:21" />
    
<meta name="content-language" content="pt" />
<meta name="viewport" content="width=device-width, initial-scale=1.0" />
<meta name="description" content="Casas para Venda, 2903 Apartamentos em Cascais, Deseja comprar casa? No maior Portal Imobiliário Nacional temos milhares de apartamentos e moradias em Lisboa, no Porto e por todo o país." />
<meta name="keywords" content="Casas para Venda, Apartamentos em Cascais, venda, compra, comprar, casas, imóveis, apartamentos, moradias, terrenos, porto, lisboa" />
<meta name="referrer" content="always" />
<link rel="alternate" t


Sabemos que las etiquetas se ponen al comienzo y al finalizar lo que va dentro de ella, y tienen una estructura como:
```
<etiqueta propiedad_etiqueta1=propiedad_1 propiedad_etiqueta2=propiedad2 ...> Cosas que van dentro de la etiqueta </etiqueta>
```

Dentro de una etiqueta puede haber otras etiquetas, ya que se sigue una estructura de árbol. Por ejemplo, la etiqueta principal que diferencia las cabeceras (información de configuración no relevante para nuestro análisis) del cuerpo de la web es ``<body></body>``, y dentro de ella estarán el resto.


Ponernos a analizar directamente este texto podría costarnos bastante, por lo que existen librerías que ya lo hacen por nosotros, como ``BeautifulSoup``. Para ello, nos creamos un objeto de esta librería a partir del texto de lo que hemos leído con ``requests``:

In [255]:
html_soup = BeautifulSoup(response.text, 'html.parser')

Ahora que tenemos el objeto, tenemos que buscar las etiquetas que nos interesen, que BeautifulSoup se encargará de buscarlas por nosotros. La sintaxis se muestra a continuación con un ejemplo.

Por ejemplo, si queremos extraer todas las etiquetas ``div`` con el atributo ``class`` igual a ``"searchResultProperty"`` (el cual será una combinación etiqueta + valor de atributo característica para lo que quiera buscar, porque ya hemos mirado el código de la web antes y hemos identificado ese patrón, que en este caso sería de cada casa), la sintaxis sería como sigue:

In [257]:
house_containers = html_soup.find_all('div', class_="searchResultProperty")

``findall()`` busca *TODAS* las coincidencias y devuelve una lista de strings con cada una de ellas. Veamos qué tenemos en la primera que nos devuelve:

In [261]:
house_containers[0]

<div class="searchResultProperty item G3Position hastitle" data-pf="65" data-pkc="false" data-uid="07792c0f-5be3-11eb-b1fa-060000000056">
<script type="application/ld+json">{"@context":"http://schema.org","@type":"Offer","image":"http://media.casasapo.pt/Z640x480/Wnone/S5/C5773/P20442246/Tphoto/ID86ec3701-0000-0500-0000-00000da4b066.jpg","name":"Apartamento T2 Areias do Murtal (Estoril), Cascais e Estoril, Cascais","category":"Apartamentos","description":"Apartamento T1+1 São João do Estoril.  Apartamento composto por:  Cozinha de aproximadamente 10m2 com varanda fechada espaçosa de 3 m2, Sala de 17m2, Hall de entrada de 1.7m2, Casa de banho com chuveiro de 4m2, Quarto (...)","price":["168 000 €"],"priceCurrency":["€"],"availableAtOrFrom":{"@type":"Place","address":{"addressCountry":"PT","addressLocality":"Cascais","addressRegion":"Cascais e Estoril"},"geo":{"@type":"GeoCoordinates","latitude":38.700868627,"longitude":-9.3724942733}},"seller":{"@context":"http://schema.org","@type":"Re

Como puedes observar, tal como hemos comentado con anterioridad, tenemos otras etiquetas dentro de nuestras etiquetas. Puede que lo que busquemos esté en un nivel más profundo, por lo que podemos hacer más "zoom" volviendo a preguntar y quedános con lo que realmente nos interesa de ese subconjunto de datos de la web:

In [262]:
first = house_containers[1]

first.find_all("span")[3].text.replace('\xa0', '')

'\r\n                        Apartamento T2, Sassoeiros, Carcavelos e Parede, Cascais, Lisboa\r\n                    '

El objeto ``first`` ahora tiene la estructura de nuestra primera casa, sacada de ``house_containers``.

Bueno, pues ahora que ya sabemos cómo manejarnos con BeautifulSoup, vamos a proceder a obtener algo de información que nos interese, como el precio de la vivienda o sus características.

### Obteniendo el precio de la vivienda

Tras analizar la web, hemos detectado que, dentro del objeto que hemos extraído en el apartado anterior, podemos identificar el precio de la vivienda en el cuarto objeto con la etiqueta ``span``:

In [264]:
var_1 = first.find_all("span")[5].text
var_1

'207\xa0000 €'

In [268]:
var_1 = var_1.replace('\xa0', '')
var_1

'207000 €'

In [269]:
# Y si solo queremos el número:
# Sacamos caracter a caracter si es número:
var_precio_char = [char for char in var_1 if char.isdigit()]
# Los juntamos en un string (porque tendremos una lista del ejercicio anterior):
var_precio_str = ''.join(var_precio_char)
# Y lo convertimos a entero:
var_precio_int = int(var_precio_str)

In [270]:
print(var_precio_int, type(var_precio_int))

207000 <class 'int'>


### Obteniendo las características de la casa

Del mismo modo que hemos visto lo anterior, podemos extraer otras características de la casa (en base a otros patrones). Para analizarlo, podemos hacer algo como lo que mostramos a continuación, que es, tras extraer qué etiquetas los contenían, inspeccionar cuántos elementos tenemos y qué contiene cada uno. A continuación, se hace para ``span``, que es una etiqueta que hemos visto (tras inspeccionar con el navegador) que tiene parámetros interesantes:

In [271]:
[span.text for span in first.find_all("span")]

['Apartamento T2 com varanda em Sassoeiros',
 '\n\n',
 '\n\n',
 '\r\n                        Apartamento T2, Sassoeiros, Carcavelos e Parede, Cascais, Lisboa\r\n                    ',
 'Contacte Anunciante',
 '207\xa0000 €']

Aquí podemos ver cosas como el precio o el nombre. Sin embargo, la mayoría de casos veremos que no siempre es todo tan bonito como parece, y algunas veces este patrón falla. Bueno, pues nuestro trabajo será identificar patrones para solucionar estos contratiempos y hacerlo lo más robusto posible. Para demostrar esto, vamos a ver unos cuántos ejemplos del span, donde detectaremos que no todos los pisos tienen el precio en la posición indicada:

In [272]:
[[span.text for span in first.find_all("span")] for first in house_containers]

[['T1 + 1 em São João do Estoril',
  '\n\n',
  '\n\n',
  '\r\n                        Apartamento T2, Areias do Murtal (Estoril), Cascais e Estoril, Lisboa\r\n                    ',
  'Contacte Anunciante',
  '168\xa0000 €'],
 ['Apartamento T2 com varanda em Sassoeiros',
  '\n\n',
  '\n\n',
  '\r\n                        Apartamento T2, Sassoeiros, Carcavelos e Parede, Cascais, Lisboa\r\n                    ',
  'Contacte Anunciante',
  '207\xa0000 €'],
 ['Monte Estoril T3 novo em Condomínio com jardim e piscina na Avenida Saboia.',
  '\r\n                        Apartamento T3, Monte Estoril, Cascais e Estoril, Lisboa\r\n                    ',
  'Contacte Anunciante',
  '1\xa0350\xa0000 €'],
 ['ESTORIL-QUINTA DA GRACIOSA apartamento T3 totalmente remodelado virado para a zona do Club, Piscina,tem 3 estacionamentos',
  '\r\n                        Apartamento T3, Quinta da Graciosa (Estoril), Cascais e Estoril, Lisboa\r\n                    ',
  'Contacte Anunciante',
  '460\xa0000 €\x

Podríamos limpiar los elementos ``\n\n`` que aparecen aleatoriamente en las listas:

In [157]:
# [[span.text for span in first.find_all("span") if span.text != '\n\n'] for first in house_containers]

In [273]:
pisos = []
propiedades = []
for first in house_containers:
    for span in first.find_all("span"):
        if span.text != '\n\n':
            propiedades.append(span.text)
    pisos.append(propiedades)
    propiedades = []
pisos

[['T1 + 1 em São João do Estoril',
  '\r\n                        Apartamento T2, Areias do Murtal (Estoril), Cascais e Estoril, Lisboa\r\n                    ',
  'Contacte Anunciante',
  '168\xa0000 €'],
 ['Apartamento T2 com varanda em Sassoeiros',
  '\r\n                        Apartamento T2, Sassoeiros, Carcavelos e Parede, Cascais, Lisboa\r\n                    ',
  'Contacte Anunciante',
  '207\xa0000 €'],
 ['Monte Estoril T3 novo em Condomínio com jardim e piscina na Avenida Saboia.',
  '\r\n                        Apartamento T3, Monte Estoril, Cascais e Estoril, Lisboa\r\n                    ',
  'Contacte Anunciante',
  '1\xa0350\xa0000 €'],
 ['ESTORIL-QUINTA DA GRACIOSA apartamento T3 totalmente remodelado virado para a zona do Club, Piscina,tem 3 estacionamentos',
  '\r\n                        Apartamento T3, Quinta da Graciosa (Estoril), Cascais e Estoril, Lisboa\r\n                    ',
  'Contacte Anunciante',
  '460\xa0000 €\xa0/\xa01\xa0500 €'],
 ['Excelente Aparta

In [275]:
propiedades = []
for span in first.find_all("span"):
    if span.text != '\n\n':
        propiedades.append(span.text)
pisos.append(propiedades)
propiedades[3]

'490\xa0000 €'

In [276]:
# [[span.text for span in first.find_all("span") if span.text != '\n\n'] for first in house_containers]

Con esto ya lo tendríamos normalizado y todo lo que extrajéramos de esta secuencia tendría la misma referencia.

Ahora, probamos con otros elementos, como ``p``:

In [287]:
[[span.text for span in first.find_all("p")] for first in house_containers]
# [len([span.text for span in first.find_all("p")]) for first in house_containers]
# Si nos fijamos, de aquí también podemos sacar el precio!!

[['\n\r\n                        Apartamento T2, Areias do Murtal (Estoril), Cascais e Estoril, Lisboa\r\n                    \n',
  '\r\n                    Areias do Murtal (Estoril), Cascais e Estoril, Lisboa\r\n                ',
  '\r\n                                comprar\r\n                            ',
  '\n168\xa0000 €\n',
  'Estado',
  'Usado',
  'Área Útil',
  '68m²',
  'Área Bruta',
  '74m²',
  'Área Terreno',
  '-',
  '\r\n                    Apartamento T1+1 São João do Estoril.  Apartamento composto por:  Cozinha de aproximadamente 10m2 com varanda fechada espaçosa de 3 m2, Sala de 17m2, Hall de entrada de 1.7m2, Casa de banho com chuveiro de 4m2, Quarto (...)\r\n                '],
 ['\n\r\n                        Apartamento T2, Sassoeiros, Carcavelos e Parede, Cascais, Lisboa\r\n                    \n',
  '\r\n                    Sassoeiros, Carcavelos e Parede, Cascais, Lisboa\r\n                ',
  '\r\n                                comprar\r\n                

### EJERCICIO

1. En base a la extracción anterior, ¿podrías extraer la información del Estado de la vivienda (en portugués, no hace falta que traduzcas)?
2. Haz lo propio para los diferentes Áreas (Bruta, Útil y Terreno)
3. ¿Podrías identificar además si se puede comprar y/o alquilar? *Tip: Puede que las viviendas que estés visualizando solo se puedan comprar, pero si buscas más ejemplos, te encontrarás alguno que también se pueda alquilar.*

Continuemos sacando los elementos que hayamos detectado tras el análisis:

In [283]:
# Location
location = first.find_all('p')[1].text
location

'\r\n                    Estoril, Cascais e Estoril, Lisboa\r\n                '

In [284]:
# Con strip eliminamos todas esas cosas raras a izquierda y derecha de nuestro string:
location.strip()

'Estoril, Cascais e Estoril, Lisboa'

In [288]:
# Tamaño en m2 (en este caso no está informado):
first.find_all('p')[7]

<p>-</p>

In [293]:
# Descripción corta:
first.find_all('p', class_="searchPropertyDescription")[0].text.strip()

'Apartamento T2 totalmente remodelado, com vista de mar, situado no Estoril. Com uma localização privilegiada a poucos minutos do Casino Estoril.  Este apartamento com 88 m2, inserido no segundo andar de um prédio sem (...)'

Estos ejemplos deberían ser suficientes para que hagamos nuestra propia investigación. El *modus operandi* se reducirá a probar con la estructura html y manipular los valores que se devuelven hasta que obtenemos lo que queremos.

### Continuamos cogiendo todos los links

Estos links nos podrían servir para seguir investigando la web, haciendo las sucesivas peticiones web.

**¡IMPORTANTE!** Hay que tener mucho cuidado con estas cosas, ya que es muy fácil entrar en un raastreo cíclico del que no podamos salir, pues casi todas las webs (al menos, si están bien diseñadas) tendrán enlaces que se hagan referencias a ellas mismas, pudiendo navegar por ellas, sí, pero facilitando que nos quedemos yendo de un sitio para otro en un bucle infinito. Por ello, en este tipo de casos, es recomendable llevar una lista con las páginas analizadas y no enviar nuevas peticiones si no es a un link nuevo (salvo en caso de que haya actualizaciones, lo que añadiría un grado más a nuestro ya de por sí complejo mecanismo).

In [297]:
first.find_all('a')[0].get('href')

'https://gespub.casa.sapo.pt/v3/webinterface/client/counter.aspx?p=692842,689428&c=1&MCA=1,1&TW=152&BU=1&l=https%3a%2f%2fcasa.sapo.pt%2fcomprar-apartamento-t2-cascais-e-estoril-estoril-b326e680-60c4-11e7-acd0-00155d01ea14.html%3fg3pid%3d692842'

In [298]:
# Para coger todos los links
for url in first.find_all('a'):
    print(url.get('href'))

https://gespub.casa.sapo.pt/v3/webinterface/client/counter.aspx?p=692842,689428&c=1&MCA=1,1&TW=152&BU=1&l=https%3a%2f%2fcasa.sapo.pt%2fcomprar-apartamento-t2-cascais-e-estoril-estoril-b326e680-60c4-11e7-acd0-00155d01ea14.html%3fg3pid%3d692842
https://gespub.casa.sapo.pt/v3/webinterface/client/counter.aspx?p=692842,689428&c=1&MCA=1,1&TW=152&BU=1&l=https%3a%2f%2fcasa.sapo.pt%2fcomprar-apartamento-t2-cascais-e-estoril-estoril-b326e680-60c4-11e7-acd0-00155d01ea14.html%3fg3pid%3d692842
/agencia/quintela-e-penalva-real-estate/?cl=11244&sys=5
https://gespub.casa.sapo.pt/v3/webinterface/client/counter.aspx?p=692842,689428&c=1&MCA=1,1&TW=152&BU=1&l=https%3a%2f%2fcasa.sapo.pt%2fcomprar-apartamento-t2-cascais-e-estoril-estoril-b326e680-60c4-11e7-acd0-00155d01ea14.html%3fg3pid%3d692842
https://gespub.casa.sapo.pt/v3/webinterface/client/counter.aspx?p=692842,689428&c=1&MCA=1,1&TW=152&BU=1&l=https%3a%2f%2fcasa.sapo.pt%2fcomprar-apartamento-t2-cascais-e-estoril-estoril-b326e680-60c4-11e7-acd0-00155d0

Si nos fijamos, todos los enlaces (salvo 1) hacen referencia al mismo sitio, que no es otro que la página dedicada a expandir la información de esa vivienda en concreto. El otro enlace, sin embargo, hace una referencia relativa a la web con la información de la inmobiliaria. Se trata de una web relativa porque se accederá a ella utilizando la url principal de la web (``https://casa.sapo.pt/``) añadiéndole esa ruta relativa (``/agencia/quintela-e-penalva-real-estate/?cl=11244&sys=5``), lo cual podría ser útil para analizar las inmobiliarias.

En este caso, haremos un ejemplo sencillo y no nos meteremos a iterar sobre los enlaces. Lo que vamos a hacer es crearnos un script que rastree 10 páginas y nos extraiga nombre, precio, zona, descriptción y url, de cada una de las viviendas: 

In [224]:
# Creamos una lista que va a formar a nuestro dataframe
titles = []
prices = []
zone = []
descriptions = []
urls = []

In [299]:
# Definimos límites de la llamada:
n_page_min = 1
n_page_max = 10

for page in range (n_page_min, n_page_max+1):
    # Para que no nos volvamos locos esperando, ya que es un proceso que puede tardar un rato, vamos a imprimir por pantalla el progreso.
    # Como son muchas páginas, y haciendo un print normal nos escribiría tantas líneas como páginas analizara, vamos a hacer algo más
    # interesante, vamos a hacer que se sobreescriba la línea. Para ello, añadimos el parámetro "end" con el valor '\r', que le dirá
    # al print que no añada el salto de línea que suele meter por defecto, haciendo que el print siguiente sobreescriba al que vemos:
    print(f"Scrapeando página {page}/{n_page_max}", end='\r')
    sapo_url = f"https://casa.sapo.pt/comprar-apartamentos/cascais/?pn={page}"
    r = get(sapo_url, headers = headers)
    page_html = BeautifulSoup(r.text, 'html.parser')
    house_containers = page_html.find_all('div', class_ = "searchResultProperty")
    
    if house_containers != []:
        for container in house_containers:
            try:
                propiedades = []
                for span in container.find_all("span"):
                    if span.text != '\n\n':
                        propiedades.append(span.text)
                pisos.append(propiedades)
                
                price = propiedades[3]
                
                # Añadimos una cosa extra, y es que pueden aparecer 2 precios (compra/alquiler). Nos quedamos con el primero. Para ello,
                # lo único que hacemos es detectar la barra separadora y quedarnos con lo primero:
                price = price.split('/')[0]
                price_ = [int(price[s]) for s in range(0, len(price)) if price[s].isdigit()]

                price = ''
                for x in price_:
                    price = price + str(x)
                prices.append(int(price))
            
            except:
                # Puede que no tenga precio. En ese caso, pasamos de ese registro
                continue
            
            #Zona
            location = container.find_all('p', class_="searchPropertyLocation")[0].text
            location = location[22:location.find(',')]
            zone.append(location)
            
            #Title
            name = container.find_all('span')[0].text
            titles.append(name)
            
            #Description
            desc = first.find_all('p', class_="searchPropertyDescription")[0].text[22:-25]
            descriptions.append(desc)
            
            #Url
            link = container.find_all('a')[0].get('href')[0:-6]
            urls.append(link)
            
    else:
        print("ERROR. Saliendo del bucle...")
        break
    
    # Hemos completado de escanear los pisos de la página actual:
    print(f"Página {page} completa.", end='\r')
    
    # Lo que estamos haciendo iteración a iteración es leer página a página, y dentro de cada página, nos quedamos con las características 
    #de las que nos interesan. En cada llamada estamos simulando un navegador, por lo que hacerlas muy de seguido podría hacer saltar las alarmas
    #y que sospecharan que somos bots o algo por el estilo, dando lugar a errores de respuesta. Para solucionar esto, se puede poner un 'sleep',
    #que lo que hará será esperar un poquito para enviar la siguiente instrucción. Aunque en este caso no será necesario:
    #sleep(randint(1,2))
    
print(f"Se han \'scrapeado\' {page} páginas, que contienen un total de {len(titles)} propiedades.")   

Se han 'scrapeado' 10 páginas, que contienen un total de 300 propiedades.


In [301]:
cols = ['Title', 'Price', 'Zone',  'Description', 'URL']

portucasas = pd.DataFrame({'Title': titles,
                          'Price': prices,
                          'Zone': zone,
                          'Description': descriptions,
                          'URL': urls})[cols]

portucasas.to_excel('portucasas.xls', index=False)
portucasas_df = pd.read_excel('portucasas.xls')

In [302]:
portucasas_df

,Title,Price,Zone,Description,URL
0,Apartamento T2 com varanda em Sassoeiros,207000,Sassoeiros,Monte Estoril apartamento T3 novo em Condomíni...,https://gespub.casa.sapo.pt/v3/webinterface/cl...
1,Monte Estoril T3 novo em Condomínio com jardim...,1350000,Monte Estoril,Monte Estoril apartamento T3 novo em Condomíni...,https://gespub.casa.sapo.pt/v3/webinterface/cl...
2,ESTORIL-QUINTA DA GRACIOSA apartamento T3 tota...,460000,Quinta da Graciosa (Estoril),Monte Estoril apartamento T3 novo em Condomíni...,https://gespub.casa.sapo.pt/v3/webinterface/cl...
3,Excelente Apartamento T5 em Condomínio,890000,Alto das Flores (Cascais),Monte Estoril apartamento T3 novo em Condomíni...,https://gespub.casa.sapo.pt/v3/webinterface/cl...
4,Apartamento T2 Recuperado no Bairro do Rosário...,390000,Bairro do Rosário (Cascais),Monte Estoril apartamento T3 novo em Condomíni...,https://gespub.casa.sapo.pt/v3/webinterface/cl...
...,...,...,...,...,...
295,Apartamento T4 em condomínio fechado com pisci...,1650000,Quinta da Marinha (Cascais),"Apartamento T2 totalmente remodelado, com vist...",/comprar-apartamento-t4-cascais-e-estoril-quin...
296,"T3 com box, Cascais",725000,Costa da Guia (Cascais),"Apartamento T2 totalmente remodelado, com vist...",/comprar-apartamento-t3-cascais-e-estoril-cost...
297,Lançamento – T2 (115m2) novo em São Domingos d...,365000,Rebelva,"Apartamento T2 totalmente remodelado, com vist...",/comprar-apartamento-t2-cascais-sao-domingos-d...
298,Apartamento T1 83m2 - Marianas Premium,300000,Quinta das Marianas (Parede),"Apartamento T2 totalmente remodelado, com vist...",/comprar-apartamento-t1-cascais-carcavelos-e-p...


### EJERCICIO

Repasa estos conceptos poniéndolos en práctica. Busca una web en la que veas la información distribuida de una forma que entiendas y puedas obtener patrones, y saca de ellos información que te ayude en un posterior estudio